<a href="https://colab.research.google.com/github/Sakin101/GANS/blob/main/DGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch as th
import numpy as np
import pandas as pd
import torchvision
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.autograd.variable import Variable
from torchvision import transforms, datasets
from torch.utils.tensorboard import SummaryWriter
device='cuda' if th.cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
class Discriminator(nn.Module):
  def __init__(self,in_feature,out_feature):
    super().__init__()
    self.dis=nn.Sequential(
        nn.Conv2d(in_feature, out_feature, 4, 2, 1, bias=False),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Conv2d(out_feature, out_feature * 2, 4, 2, 1, bias=False),
        nn.BatchNorm2d(out_feature * 2),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Conv2d(out_feature * 2, out_feature * 4, 4, 2, 1, bias=False),
        nn.BatchNorm2d(out_feature * 4),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Conv2d(out_feature * 4, out_feature * 8, 4, 2, 1, bias=False),
        nn.BatchNorm2d(out_feature * 8),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Conv2d(out_feature * 8, 1, 4, 1, 0, bias=False),
        nn.Sigmoid()
    )
  def forward(self,x):
    return self.dis(x)

class Generator(nn.Module):
  def __init__(self,noise_dim,in_feature,out_feature,img_channel):
      super(Generator,self).__init__()
      self.gene=nn.Sequential(
          nn.ConvTranspose2d(in_feature,out_feature*8,4,1,0),
          nn.BatchNorm2d(out_feature*8),
          nn.ReLU(),
          nn.ConvTranspose2d(out_feature*8,out_feature*4,4,2,1),
          nn.BatchNorm2d(out_feature*4),
          nn.ReLU(),
          nn.ConvTranspose2d(out_feature*4,out_feature*2,4,2,1),
          nn.BatchNorm2d(out_feature*2),
          nn.ReLU(),
        nn.ConvTranspose2d(out_feature*2,out_feature,4,2,1),
          nn.BatchNorm2d(out_feature),
          nn.ReLU(),
          nn.ConvTranspose2d(out_feature,1,4,2,1),
          nn.Tanh()
          # self.get_block(100,out_feature*8,1,4),
          # self.get_block(out_feature*8,out_feature*4,2,4,1,False),
          # self.get_block(out_feature*4,out_feature*2,2,4,1,False),
          # self.get_block(out_feature*2,out_feature,  2,4,1,False),
          # self.get_block(out_feature,img_channel,2,4,1,True),

      )
  def get_block(in_feature,out_feature,stride,kernel,padding=0,final_layer=False):
    if(not final_layer):
      return nn.Sequential(
          nn.ConvTranspose2d(in_feature,out_feature,kernel,stride,padding),
          nn.BatchNorm2d(out_feature),
          nn.LeakyReLU(0.2)
      )
    else:
      return nn.Sequential(
          nn.ConvTranspose2d(in_feature,out_feature,kernel,stride,padding),
          nn.Tanh()
      )
  def forward(self,x):
      return self.gene(x)




In [ ]:
transform=transforms.Compose([
                              transforms.Resize(64),

                               transforms.ToTensor(),

                               transforms.Normalize((0.5),(0.5)),
                           ])
datasets=torchvision.datasets.CelebA(root='datasets/',transform=transform,download=True)


RuntimeError: ignored

In [ ]:
lr=0.0002
z_dim=100
img_dim=28*28
discriminator_loss=nn.BCELoss()
generator_loss=nn.L1Loss()
batch_size=32

disc=Discriminator(1,64).to(device)
gen=Generator(z_dim,100,64,1).to(device)


noise=th.randn(batch_size,z_dim).to(device)
dis_optim=th.optim.Adam(disc.parameters(),lr,betas=(0.5, 0.9))
gen_optim=th.optim.Adam(gen.parameters(),lr,betas=(0.5, 0.9))

loader=DataLoader(datasets,batch_size=batch_size)
print(loader)
epoch=5
step=0
writer_fake = SummaryWriter(f"logs/fake")
writer_real = SummaryWriter(f"logs/real")

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs/fashion_mnist_experiment_1')
for num_epochs in range(epoch):
  for ix,(real,lable) in enumerate(loader):
    real=real.to(device)
    batch_size=real.shape[0]
    noise=th.randn(batch_size,z_dim,1,1).to(device)
    fake_img=gen(noise)
    disc_fake=disc(fake_img).view(-1)
    gen_loss=discriminator_loss(disc_fake,th.zeros_like(disc_fake))
    real_disc=disc(real)
    real_loss=discriminator_loss(real_disc,th.ones_like(real_disc))

    total_loss=(real_loss+gen_loss)/2
    disc.zero_grad()
    total_loss.backward(retain_graph=True)
    dis_optim.step()

    output=disc(fake_img)
    gen_loss=discriminator_loss(output,th.ones_like(output))
    gen.zero_grad()
    gen_loss.backward()
    gen_optim.step()

    if ix == 0:
            print(
                f"Epoch [{epoch}/{num_epochs}] Batch {id}/{len(loader)} \
                      Loss D: {total_loss:.4f}, loss G: {gen_loss:.4f}"
            )

            with th.no_grad():
                fake = gen(noise)
                img_grid_fake = torchvision.utils.make_grid(fake, normalize=True)


                writer_fake.add_image(
                    "Mnist Fake Images", img_grid_fake, global_step=step
                )

                step += 1



In [ ]:
%load_ext tensorboard

%tensorboard --logdir /content/logs/fake
